# 用SQL語法來理解pandas

由於大多數學習使用pandas的許多學員對SQL都有一定的了解，因此以下提供一些範例來說明如何使用pandas來執行各種SQL的操作。

按照慣例，我們按以下方式導入pandas和NumPy：

In [1]:
import pandas as pd
import numpy as np

大多數範例將利用在一個範例數據集`tips.csv`。我們將數據讀入一個名為`tips`的DataFrame中，並假定我們具有相同名稱和結構的數據庫資料表。

* total_bill 帳單總金額
* tip 小費
* sex 姓別
* smoker 是否有吸煙
* day 星期幾
* time 午餐/晚餐
* size 用餐人數

In [3]:
tips = pd.read_csv("data/tips.csv")

print(tips.info())

print(tips.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 13.5+ KB
None
   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


## SELECT

在SQL中，`SELECT`是使用你要選擇的列的逗號分隔列表（或使用*來選擇所有列）來完成的：

```sql
SELECT total_bill, tip, smoker, time
FROM tips
LIMIT 5;
```

對於pandas，通過將column names列表傳遞給DataFrame來完成列選擇：

In [5]:
results = tips[["total_bill", "tip", "smoker", "time"]].head(5)

print(results)

print(type(results))

   total_bill   tip smoker    time
0       16.99  1.01     No  Dinner
1       10.34  1.66     No  Dinner
2       21.01  3.50     No  Dinner
3       23.68  3.31     No  Dinner
4       24.59  3.61     No  Dinner
<class 'pandas.core.frame.DataFrame'>


調用不帶列名列表的DataFrame會顯示所有列（類似於SQL的 *）。

在SQL中，你可以添加一個計算列：

```sql
SELECT *, tip/total_bill as tip_rate
FROM tips
LIMIT 5;
```

對於pandas，可以使用DataFrame的`DataFrame.assign()`方法增加一個新列：

In [6]:
results = tips.assign(tip_rate=tips["tip"]/tips["total_bill"])

print(results.info())

print(results.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
 7   tip_rate    244 non-null    float64
dtypes: float64(3), int64(1), object(4)
memory usage: 15.4+ KB
None
   total_bill   tip     sex smoker  day    time  size  tip_rate
0       16.99  1.01  Female     No  Sun  Dinner     2  0.059447
1       10.34  1.66    Male     No  Sun  Dinner     3  0.160542
2       21.01  3.50    Male     No  Sun  Dinner     3  0.166587
3       23.68  3.31    Male     No  Sun  Dinner     2  0.139780
4       24.59  3.61  Female     No  Sun  Dinner     4  0.146808


## WHERE

通過`WHERE`子句在SQL中對數據進行過濾(filtering)。

```sql
SELECT *
FROM tips
WHERE time = 'Dinner'
LIMIT 5;
```

DataFrame可以通過多種方式進行過濾。最直觀的是使用[boolean indexing](https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-boolean)。

In [7]:
results = tips[tips["time"] == "Dinner"]

print(results.info())

print(results.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  176 non-null    float64
 1   tip         176 non-null    float64
 2   sex         176 non-null    object 
 3   smoker      176 non-null    object 
 4   day         176 non-null    object 
 5   time        176 non-null    object 
 6   size        176 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 11.0+ KB
None
   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


上面的語法只是將一個`Series`的True/False傳遞給DataFrame，並返回所有帶有True的row。

In [8]:
is_dinner = tips["time"] == "Dinner"

print(type(is_dinner))

print(is_dinner)

print(is_dinner.value_counts()) # 透過 value_counts()來取得取的統計

<class 'pandas.core.series.Series'>
0      True
1      True
2      True
3      True
4      True
       ... 
239    True
240    True
241    True
242    True
243    True
Name: time, Length: 244, dtype: bool
True     176
False     68
Name: time, dtype: int64


就像SQL的`OR`和`AND`一樣，可以使用`|`(OR) 與 `&` (AND)將多個條件傳遞給DataFrame。

```sql
-- 小費大於 $5.00 並且用餐時間是 'Dinner' 
SELECT *
FROM tips
WHERE time = 'Dinner' AND tip > 5.00;
```


In [9]:
# tips of more than $5.00 at Dinner meals
results = tips[(tips["time"]=="Dinner") & (tips["tip"] > 5.00)]

print(results.info())

print(results.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 23 to 239
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  15 non-null     float64
 1   tip         15 non-null     float64
 2   sex         15 non-null     object 
 3   smoker      15 non-null     object 
 4   day         15 non-null     object 
 5   time        15 non-null     object 
 6   size        15 non-null     int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 960.0+ bytes
None
    total_bill   tip     sex smoker  day    time  size
23       39.42  7.58    Male     No  Sat  Dinner     4
44       30.40  5.60    Male     No  Sun  Dinner     4
47       32.40  6.00    Male     No  Sun  Dinner     4
52       34.81  5.20  Female     No  Sun  Dinner     4
59       48.27  6.73    Male     No  Sat  Dinner     4


```sql
-- 用餐人數至少有 5人或是總金額大於 45
SELECT *
FROM tips
WHERE size >= 5 OR total_bill > 45;
```

In [10]:
results = tips[(tips["size"] >= 5) | (tips["total_bill"] > 45)]

print(results.info())

print(results.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 59 to 216
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  13 non-null     float64
 1   tip         13 non-null     float64
 2   sex         13 non-null     object 
 3   smoker      13 non-null     object 
 4   day         13 non-null     object 
 5   time        13 non-null     object 
 6   size        13 non-null     int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 832.0+ bytes
None
     total_bill   tip     sex smoker   day    time  size
59        48.27  6.73    Male     No   Sat  Dinner     4
125       29.80  4.20  Female     No  Thur   Lunch     6
141       34.30  6.70    Male     No  Thur   Lunch     6
142       41.19  5.00    Male     No  Thur   Lunch     5
143       27.05  5.00  Female     No  Thur   Lunch     6


NULL檢查是使用`notna()`和`isna()`方法完成的。

In [11]:
df_tmp = pd.DataFrame(
     {
         "col1": ["A", "B", np.NaN, "C", "D"], 
         "col2": ["F", np.NaN, "G", "H", "I"]
     }
)

print(df_tmp.info())
print(df_tmp.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   col1    4 non-null      object
 1   col2    4 non-null      object
dtypes: object(2)
memory usage: 208.0+ bytes
None
  col1 col2
0    A    F
1    B  NaN
2  NaN    G
3    C    H
4    D    I


假設我們有一個與上面的DataFrame具有相同結構的資料表。通過以下查詢，我們只能看到col2為NULL的記錄：

```sql
SELECT *
FROM frame
WHERE col2 IS NULL;
```

In [12]:
results = df_tmp[df_tmp["col2"].isna()]

print(results.head())

  col1 col2
1    B  NaN


可以使用`notna()`來獲取`col1`不為`null`的資料。

```sql
SELECT *
FROM frame
WHERE col1 IS NOT NULL;
```

In [14]:
results = df_tmp[df_tmp["col1"].notna()]

print(results.head())

  col1 col2
0    A    F
1    B  NaN
3    C    H
4    D    I


## GROUP BY

在pandas中，SQL的`GROUP BY`操作是使用類似命名的`groupby()`方法執行的。 `groupby()`通常是指一個過程，我們希望將數據集劃分為多個組，應用某些功能函數（通常是aggregation）。

常見的SQL操作是獲取整個數據集中每個組中的記錄數。例如，通過查詢可以了解不同性別會給予的小費人數：

```sql
SELECT sex, count(*)
FROM tips
GROUP BY sex;
/*
Female     87
Male      157
*/
```

在pandas裡可以這麼做:

In [16]:
results = tips.groupby("sex").size()

print(results)

print(type(results))

sex
Female     87
Male      157
dtype: int64
<class 'pandas.core.series.Series'>


注意，在以上的範例中，我們使用`size()`而不是`count()`。這是因為`count()`函數會返回每個column中`not null`的資料筆數。

In [17]:
results = tips.groupby("sex").count()

print(results)

print(type(results))

        total_bill  tip  smoker  day  time  size
sex                                             
Female          87   87      87   87    87    87
Male           157  157     157  157   157   157
<class 'pandas.core.frame.DataFrame'>


或者，我們可以將`count()`方法應用於指定的column：

In [18]:
results = tips.groupby("sex")["total_bill"].count()

print(results)

print(type(results))

sex
Female     87
Male      157
Name: total_bill, dtype: int64
<class 'pandas.core.series.Series'>


當然我們也可以一次地套用多種functions。舉例來說，假設我們想了解小費金額在一周中的一天之間有何不同-`agg()`可讓我們達到這個目的。

```sql
SELECT day, AVG(tip), COUNT(*)
FROM tips
GROUP BY day;
/*
Fri   2.734737   19
Sat   2.993103   87
Sun   3.255132   76
Thur  2.771452   62
*/
```

In [19]:
results = tips.groupby("day").agg({"tip":np.mean, "day": np.size})

print(results)

print(type(results))

           tip  day
day                
Fri   2.734737   19
Sat   2.993103   87
Sun   3.255132   76
Thur  2.771452   62
<class 'pandas.core.frame.DataFrame'>


通過將多個列的列表傳遞給`groupby()`方法，可以完成對多個列的分組。

```sql
SELECT smoker, day, COUNT(*), AVG(tip)
FROM tips
GROUP BY smoker, day;
/*
smoker day
No     Fri      4  2.812500
       Sat     45  3.102889
       Sun     57  3.167895
       Thur    45  2.673778
Yes    Fri     15  2.714000
       Sat     42  2.875476
       Sun     19  3.516842
       Thur    17  3.030000
*/
```

In [21]:
results = tips.groupby(["smoker", "day"]).agg({"tip": [np.size, np.mean]})

print(results)

print(type(results))

              tip          
             size      mean
smoker day                 
No     Fri    4.0  2.812500
       Sat   45.0  3.102889
       Sun   57.0  3.167895
       Thur  45.0  2.673778
Yes    Fri   15.0  2.714000
       Sat   42.0  2.875476
       Sun   19.0  3.516842
       Thur  17.0  3.030000
<class 'pandas.core.frame.DataFrame'>


## JOIN

可以使用`join()`或`merge()`來執行JOIN。預設情況下，`join()`將在其索引上聯接DataFrame。透過參數設定可讓你指定要執行JOIN的類型（LEFT，RIGHT，INNER，FULL）或要聯接的列（列名或索引）。

In [22]:
# generate two dataframe
df1 = pd.DataFrame({"key": ["A", "B", "C", "D"], "value": np.random.randn(4)})

df2 = pd.DataFrame({"key": ["B", "D", "D", "E"], "value": np.random.randn(4)})

print(df1)

print(df2)

  key     value
0   A -0.476651
1   B -0.908579
2   C -0.822806
3   D  1.052995
  key     value
0   B  0.000644
1   D -0.018976
2   D -0.489452
3   E -0.063446


假設我們有兩個數據庫資料表，它們的名稱和結構與我們創建的DataFrames相同。

現在，讓我們看一下各種類型的JOIN。

### INNER JOIN

```sql
SELECT *
FROM df1
INNER JOIN df2
  ON df1.key = df2.key;
```

In [25]:
# merge performs an INNER JOIN by default
results = pd.merge(df1, df2, on="key")

print(results)

print(type(results))

  key   value_x   value_y
0   B -0.908579  0.000644
1   D  1.052995 -0.018976
2   D  1.052995 -0.489452
<class 'pandas.core.frame.DataFrame'>


`merge()`還提供了一些參數，用於將一個DataFrame的列與另一個DataFrame的索引連接在一起的情況。

In [27]:
indexed_df2 = df2.set_index("key")

results = pd.merge(df1, indexed_df2, left_on="key", right_index=True)

print(results)

print(type(results))

  key   value_x   value_y
1   B -0.908579  0.000644
3   D  1.052995 -0.018976
3   D  1.052995 -0.489452
<class 'pandas.core.frame.DataFrame'>


### LEFT OUTER JOIN

```sql
-- show all records from df1
SELECT *
FROM df1
LEFT OUTER JOIN df2
  ON df1.key = df2.key;
```

In [29]:
# show all records from df1
results = pd.merge(df1, df2, on="key", how="left")

print(results)

print(type(results))

  key   value_x   value_y
0   A -0.476651       NaN
1   B -0.908579  0.000644
2   C -0.822806       NaN
3   D  1.052995 -0.018976
4   D  1.052995 -0.489452
<class 'pandas.core.frame.DataFrame'>


### RIGHT JOIN

```sql
-- show all records from df2
SELECT *
FROM df1
RIGHT OUTER JOIN df2
  ON df1.key = df2.key;
```

In [31]:
# show all records from df2
results = pd.merge(df1, df2, on="key", how="right")

print(results)

print(type(results))

  key   value_x   value_y
0   B -0.908579  0.000644
1   D  1.052995 -0.018976
2   D  1.052995 -0.489452
3   E       NaN -0.063446
<class 'pandas.core.frame.DataFrame'>


### FULL JOIN

pandas還允許FULL JOIN，它們顯示數據集的兩側，無論連接的列是否找到匹配項。其實並非所有RDBMS（MySQL）都支持FULL JOIN。

```sql
SELECT *
FROM df1
FULL OUTER JOIN df2
  ON df1.key = df2.key;
```

In [33]:
# show all records from both frames
results = pd.merge(df1, df2, on="key", how="outer")

print(results)

print(type(results))

  key   value_x   value_y
0   A -0.476651       NaN
1   B -0.908579  0.000644
2   C -0.822806       NaN
3   D  1.052995 -0.018976
4   D  1.052995 -0.489452
5   E       NaN -0.063446
<class 'pandas.core.frame.DataFrame'>


## UNION

在pandas中可以透過`concat()`來模擬SQL的`UNION ALL`。

In [35]:
df1 = pd.DataFrame(
    {"city": ["Chicago", "San Francisco", "New York City"], "rank": range(1, 4)}
)

df2 = pd.DataFrame(
    {"city": ["Chicago", "Boston", "Los Angeles"], "rank": [1, 4, 5]}
)

```sql
SELECT city, rank
FROM df1
UNION ALL
SELECT city, rank
FROM df2;
/*
         city  rank
      Chicago     1
San Francisco     2
New York City     3
      Chicago     1
       Boston     4
  Los Angeles     5
*/
```


In [38]:
results = pd.concat([df1, df2])

print(results)

print(type(results))

            city  rank
0        Chicago     1
1  San Francisco     2
2  New York City     3
0        Chicago     1
1         Boston     4
2    Los Angeles     5
<class 'pandas.core.frame.DataFrame'>


SQL的`UNION`與`UNION ALL`類似，但是`UNION`會刪除重複的行。

```sql
SELECT city, rank
FROM df1
UNION
SELECT city, rank
FROM df2;
-- notice that there is only one Chicago record this time
/*
         city  rank
      Chicago     1
San Francisco     2
New York City     3
       Boston     4
  Los Angeles     5
*/
```

在pandas中，可以將`concat()`與`drop_duplicates()`結合使用。

In [41]:
results = pd.concat([df1, df2]).drop_duplicates()

print(results)

print(type(results))

            city  rank
0        Chicago     1
1  San Francisco     2
2  New York City     3
1         Boston     4
2    Los Angeles     5
<class 'pandas.core.frame.DataFrame'>


## TOP n ROWS with OFFSET

```sql
-- MySQL
SELECT * FROM tips
ORDER BY tip DESC
LIMIT 10 OFFSET 5;
```


In [43]:
# nlargest()
# Return the first n rows ordered by columns in descending order.
results = tips.nlargest(10 + 5, columns="tip").tail(10)

print(results)

print(type(results))

     total_bill   tip     sex smoker   day    time  size
183       23.17  6.50    Male    Yes   Sun  Dinner     4
214       28.17  6.50  Female    Yes   Sat  Dinner     3
47        32.40  6.00    Male     No   Sun  Dinner     4
239       29.03  5.92    Male     No   Sat  Dinner     3
88        24.71  5.85    Male     No  Thur   Lunch     2
181       23.33  5.65    Male    Yes   Sun  Dinner     2
44        30.40  5.60    Male     No   Sun  Dinner     4
52        34.81  5.20  Female     No   Sun  Dinner     4
85        34.83  5.17  Female     No  Thur   Lunch     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
<class 'pandas.core.frame.DataFrame'>


## UPDATE

```sql
UPDATE tips
SET tip = tip*2
WHERE tip < 2;
```


In [45]:
# before UPDATE
print(tips)

# UPDATE
tips.loc[tips["tip"] < 2, "tip"] *= 2

# after UPDATE
print(tips)

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]
     total_bill   tip     sex smoker   day    time  size
0         16.99  2.02  Female     No   Sun  Dinner     2
1         10.34  3.32    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4      

## DELETE

```sql
DELETE FROM tips
WHERE tip > 9;
```

在pandas，我們的做法是"SELECT"應保留的rows成為一個新的dataframe物件。

In [47]:
tips = tips.loc[tips["tip"] <= 9]

print(results)

print(results.info())

     total_bill   tip     sex smoker   day    time  size
183       23.17  6.50    Male    Yes   Sun  Dinner     4
214       28.17  6.50  Female    Yes   Sat  Dinner     3
47        32.40  6.00    Male     No   Sun  Dinner     4
239       29.03  5.92    Male     No   Sat  Dinner     3
88        24.71  5.85    Male     No  Thur   Lunch     2
181       23.33  5.65    Male    Yes   Sun  Dinner     2
44        30.40  5.60    Male     No   Sun  Dinner     4
52        34.81  5.20  Female     No   Sun  Dinner     4
85        34.83  5.17  Female     No  Thur   Lunch     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 183 to 211
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  10 non-null     float64
 1   tip         10 non-null     float64
 2   sex         10 non-null     object 
 3   smoker      10 non-null     object 
 4   day         10 no

# 數據導入與轉出 (Imports and Exports)

> * 導入JSON數據
> * 展平嵌套集合的數據
> * 從某個網站下載CSV
> * 讀取和寫出Excel工作表

數據集(Dataset)有多種文件格式：逗號分隔值(CSV)，TAB字符分隔值(TSV)，Excel工作表(XLSX)等。某些數據格式(例如JavaScript Object Notation - JSON)並不以表格格式來存儲數據。而是將相關數據的集合嵌套在鍵值存儲中。比較下面的兩個範例：第一個範例將數據存儲在表中，第二個範例將相同的數據存儲在Python字典中。 Python的字典是鍵值數據結構的完美範例。

![](images/12_01.png)

具有相同數據的Python字典(dict)物件:

```python
{
    2000: [
        {
            "Award": "Best Actor",
            "Winner": "Russell Crowe"
        },
        {
            "Award": "Best Actress",
            "Winner": "Julia Roberts"
        }
    ],
    2001: [
        {
            "Award": "Best Actor",
            "Winner": "Denzel Washington"
        },
        {
            "Award": "Best Actress",
            "Winner": "Halle Berry"
        }
    ]
}
```


Pandas附帶有實用的utility函式，可將以`鍵值`格式存儲的數據處理成`表格`形式，反之亦然。很多時候，`數據導入`是數據分析中最具挑戰性的部分。一旦將數據保存在`DataFrame`中，就可以對其應用所有Pandas己經內置的數據處理函式來進行數據的運算與轉置。但是，要將數據轉換成為正確的格式與形狀可能有許多的挑戰。

在教程中，我們還將學習如何使用Pandas將數據結構導出為各種文件類型和數據結構。讓我們以當今最流行的數據存儲格式之一JSON開始。

##  讀取和寫入JSON文件

JavaScript Object Notation (JSON)是一種用於存儲和傳輸文字(文本)數據的格式。儘管其語法受JavaScript程式語言的啟發，但`JSON`本身與語言無關。如今，包括Python在內的大多數語言都可以生成和解析`JSON`。

JSON由`鍵值對`組成，其中`鍵`用作`值`的唯一標識符。冒號（：）將鍵連接到值。`鍵`必須是`字符串`。`值`可以是任何`有效的數據類型`，包括strings，numbers，boolean等。 JSON與Python的`字典對象 (dict)`相似。

JSON是許多現代API常用的響應格式。來自API的原始JSON響應看起來像一個純字符串。

```json
{"name":"Harry Potter","age":17,"wizard":true}
```

透過一些pretty print的工具來讓JSON數據容易閱讀:

```json
{
    "name": "Harry Potter",
    "age": 17,
    "wizard": true,
}
```

JSON數據包含三個鍵值對(key-value pairs)。

鍵值也可以指向一個陣列，即相當於Python的list物件。見下一個JSON範例中的`friends`鍵映射到具有兩個字串的list物件。

```json
{
   "name": "Harry Potter",
   "age": 17,
   "wizard": true,
   "friends": ["Ron Weasley", "Hermione Granger"],
}
```


JSON可以在嵌套物件（例如下面的`address`）中存儲其他鍵/值對。可以將其視為嵌套在另一個字典中的字典。

```json
{
   "name": "Harry Potter",
   "age": 17,
   "wizard": true,
   "friends": ["Ron Weasley", "Hermione Granger"],
   "address": {
       "street": "4 Privet Drive",
       "town": "Little Whinging"
   }
}
```


### 將JSON文件加載到DataFrame中

我們可以將JSON存儲在檔案擴展名為.json的純文本文件中。

範例數據: `prizes.json`檔案是來自諾貝爾獎API的JSON響應。它包含一個可追溯到1901年的諾貝爾獎獲得者列表。

API: http://api.nobelprize.org/v1/prize.json

以下是這個JSON檔案的預覽：

```json
{
  "prizes": [
    {
      "year": "2019",
      "category": "chemistry",
      "laureates": [
        {
          "id": "976",
          "firstname": "John",
          "surname": "Goodenough",
          "motivation": "\"for the development of lithium-ion batteries\"",
          "share": "3"
        },
        {
          "id": "977",
          "firstname": "M. Stanley",
          "surname": "Whittingham",
          "motivation": "\"for the development of lithium-ion batteries\"",
          "share": "3"
        },
        {
          "id": "978",
          "firstname": "Akira",
          "surname": "Yoshino",
          "motivation": "\"for the development of lithium-ion batteries\"",
          "share": "3"
        }
      ]
    },
    ...
    ...
}
```


#### 數據結構解析

這個JSON數據包括了一個level#1的鍵`prizes`映射到一個物件的列表:

![](images/12_03.png)

每一個在`prizes`的物件也是一個dict物件, 包括了以下的鍵值對(key-value pairs):
* year - 年份
* category - 類別
* laureates - 得獎人

![](images/12_04.png)

由於同一個諾貝爾獎可能由多個人一同取得, 因此`laureates`的值也是一個物件的列表:
* id - 內部編號
* firstname - 名
* surname - 姓
* motivation - 動機
* share - 多少人共享這個奨項

![](images/12_05.png)

Pandas的數據導入函數具有相似的命名模版。它們由`read_前綴`及其後的`文件類型`組成。例如，之前範例多次使用到`read_csv()`函數。要導入JSON文件，我們將使用`read_json()`函數。

In [1]:
import pandas as pd

nobel = pd.read_json('data/nobel.json')

print(nobel.info())

print(nobel.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   prizes  646 non-null    object
dtypes: object(1)
memory usage: 5.2+ KB
None
                                              prizes
0  {'year': '2019', 'category': 'chemistry', 'lau...
1  {'year': '2019', 'category': 'economics', 'lau...
2  {'year': '2019', 'category': 'literature', 'la...
3  {'year': '2019', 'category': 'peace', 'laureat...
4  {'year': '2019', 'category': 'physics', 'overa...


我們已經成功地將文件導入成為`dataframe`物件了，但是這種格式對於分析來說並不理想。 Pandas識別了JSON中的top-level的鍵值`prizes`，並將其設置為列名。並把每個鍵映射的值創建成Python的dict物件。

In [2]:
# 從 dataframe 中取出第一筆
first_obj = nobel.loc[0, "prizes"]

# 檢查物件的類型
print(type(first_obj))

# 打印 dict 物件的 keys
print(first_obj.keys())

<class 'dict'>
dict_keys(['year', 'category', 'laureates'])


為了有效地使用這個數據，我們需要提取top-level的鍵值對(`year`, `category`)成為`DataFrame`獨立的列。此外，我們需要讀取獲獎者(`laureates`)列表中的每個dict物件典並提取嵌套在內的一些資訊。我們的目標是為每個諾貝爾獎獲得者有個自的row，並與他們的年份`year`和類別`category`聯繫起來。理想的DataFrame形狀如下所示：

idx|id           | firstname  | surname | motivation | share | year | category
---|:-----------:|:-----------|:--------|:-----------|------:|:----:|:---------:
0    | 976 |  John |    Goodenough   | for the develop... | 3 | 2019 | chemistry
1    | 977 |  M. Stanley |    Whittingham     | for the develop... | 3 | 2019 | chemistry
2    | 978 |  Akira       |    Yoshino     | for the develop... | 3 | 2019 | chemistry

將嵌套的數據記錄提取到單個一維列表中被稱為展平(flattening)或正規化(normalizing)。 pandas包含一個內置的`json_normalize()`函數來處理繁雜的工作。讓我們在Nobel DataFrame的範例中進行嘗試。我們將使用loc訪問器訪問第一行的字典並將其分配給`chemistry_2019`變量。

In [3]:
# 取得某一個column中的特定cell
chemistry_2019 = nobel.loc[0, "prizes"]

print(type(chemistry_2019))

print("")
print("一般的打印:")
print(chemistry_2019)

# 引入pprint的套件來幫助打印 human readable的格式
import pprint

print("")
print("美化後的打印:")
pprint.pprint(chemistry_2019)

<class 'dict'>

一般的打印:
{'year': '2019', 'category': 'chemistry', 'laureates': [{'id': '976', 'firstname': 'John', 'surname': 'Goodenough', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}, {'id': '977', 'firstname': 'M. Stanley', 'surname': 'Whittingham', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}, {'id': '978', 'firstname': 'Akira', 'surname': 'Yoshino', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}]}

美化後的打印:
{'category': 'chemistry',
 'laureates': [{'firstname': 'John',
                'id': '976',
                'motivation': '"for the development of lithium-ion batteries"',
                'share': '3',
                'surname': 'Goodenough'},
               {'firstname': 'M. Stanley',
                'id': '977',
                'motivation': '"for the development of lithium-ion batteries"',
                'share': '3',
                'surname': 'Whittingham'},
               {'fir

讓我們將`chemistry_2019`字典物件傳遞給`json_normalize`函數的`data`參數。 Pandas會提取三個top-level的字典鍵("`year`", "`category`"與"`laureates`")來成為DataFrame中的列。不幸的是，該函式庫沒有將獲獎者`laureates`列表中的嵌套dict物件也轉換成單獨的列。

In [4]:
df_tmp = pd.json_normalize(data = chemistry_2019)

print(df_tmp.info())

print(df_tmp.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   year       1 non-null      object
 1   category   1 non-null      object
 2   laureates  1 non-null      object
dtypes: object(3)
memory usage: 152.0+ bytes
None
   year   category                                          laureates
0  2019  chemistry  [{'id': '976', 'firstname': 'John', 'surname':...


我們可以利用`record_path`參數來正規化嵌套的`獲獎者laureates`。我們給予一個字串，該字串指示字典dict中的哪個`鍵`包含嵌套記錄。

In [5]:
df_tmp = pd.json_normalize(data = chemistry_2019, record_path= 'laureates')

print(df_tmp.info())

print(df_tmp.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          3 non-null      object
 1   firstname   3 non-null      object
 2   surname     3 non-null      object
 3   motivation  3 non-null      object
 4   share       3 non-null      object
dtypes: object(5)
memory usage: 248.0+ bytes
None
    id   firstname      surname  \
0  976        John   Goodenough   
1  977  M. Stanley  Whittingham   
2  978       Akira      Yoshino   

                                       motivation share  
0  "for the development of lithium-ion batteries"     3  
1  "for the development of lithium-ion batteries"     3  
2  "for the development of lithium-ion batteries"     3  


現在Dataframe裡頭有我們想要的嵌套`獲獎者laureates`的一些欄位資訊了，但是現在我們也失去了原始的`year`與`categroy`列。為了保留這些top-level的鍵值對，我們可以將帶有其名稱的列表傳遞給`meta`參數。

In [6]:
df_tmp = pd.json_normalize(data=chemistry_2019, record_path="laureates", meta=["year", "category"])

print(df_tmp.info())

print(df_tmp.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          3 non-null      object
 1   firstname   3 non-null      object
 2   surname     3 non-null      object
 3   motivation  3 non-null      object
 4   share       3 non-null      object
 5   year        3 non-null      object
 6   category    3 non-null      object
dtypes: object(7)
memory usage: 296.0+ bytes
None
    id   firstname      surname  \
0  976        John   Goodenough   
1  977  M. Stanley  Whittingham   
2  978       Akira      Yoshino   

                                       motivation share  year   category  
0  "for the development of lithium-ion batteries"     3  2019  chemistry  
1  "for the development of lithium-ion batteries"     3  2019  chemistry  
2  "for the development of lithium-ion batteries"     3  2019  chemistry  


我們的正規化策略(normalization strategy)已經成功地應用在單一的`prizes`字典物件。幸運的是`json_normalize`函數可以接受一個`Series`的字典物件並element-wise重複執行資料轉換提取的邏輯。

In [7]:
# nobel 是之前步驟所創構的 dataframe 物件

df_tmp = pd.json_normalize(
    data = nobel["prizes"],
    record_path= "laureates",
    meta = ["year", "category"],
)

KeyError: 'laureates'

不幸的是，Pandas引發了`KeyError`異常。在`prizes`的`Series`物件中某些詞典dict物件中並沒有`laureates`的鍵。`json_normalize`函數無法從不存在的列表中提取嵌套的獲獎者信息。

這種問題的一種解決方法是找出這些缺少`laureates`的記錄，然後手動將一個空列表的值分配給它們。

讓我們花點時間回顧一下Python字典上的setdefault方法。考慮下面的字典。

In [8]:
cheese_consumption = {
    "France": 57.9,
    "Germany": 53.2,
    "Luxembourg": 53.2
}

`setdefault`方法將鍵值對設定在字典(dict)物件上。如果`鍵`確實存在，則該方法返回其現有值。

下面的範例嘗試將鍵`France`添加到值為`100`的`cheese_consumption`字典物件中。由於該鍵已存在，因此沒有任何更改。 Python保留原始值57.9。

In [9]:
cheese_consumption.setdefault("France", 100)

print(cheese_consumption["France"])

57.9


相比之下，下一個範例使用參數`Italy`調用`setdefault`。字典中不存在鍵`Italy`，因此Python對其進行了添加並將其賦值為48。

In [10]:
cheese_consumption.setdefault("Italy", 48)

print(cheese_consumption["Italy"])

print(cheese_consumption)

48
{'France': 57.9, 'Germany': 53.2, 'Luxembourg': 53.2, 'Italy': 48}


讓我們將此技巧應用於`prizes`中的每個嵌套字典物件。如果字典沒有`prizes`鍵，我們將使用`setdefault`方法將其添加為空列表。我們要應用`apply`方法在每個Series元素上單獨進行迭代。

In [11]:
# 定義一個function來對應用到每一個元素
def add_laureates_key(entry):
    entry.setdefault("laureates", [])
    
# 缺失值的處理
nobel["prizes"].apply(add_laureates_key)

print(nobel["prizes"])

0      {'year': '2019', 'category': 'chemistry', 'lau...
1      {'year': '2019', 'category': 'economics', 'lau...
2      {'year': '2019', 'category': 'literature', 'la...
3      {'year': '2019', 'category': 'peace', 'laureat...
4      {'year': '2019', 'category': 'physics', 'overa...
                             ...                        
641    {'year': '1901', 'category': 'chemistry', 'lau...
642    {'year': '1901', 'category': 'literature', 'la...
643    {'year': '1901', 'category': 'peace', 'laureat...
644    {'year': '1901', 'category': 'physics', 'laure...
645    {'year': '1901', 'category': 'medicine', 'laur...
Name: prizes, Length: 646, dtype: object


現在，所有嵌套詞典都有一個`laurates`鍵，我們可以重新調用`json_normalize`函數。

In [12]:
winners = pd.json_normalize(
             data = nobel["prizes"],
             record_path = "laureates",
             meta = ["year", "category"]
         )

print(winners.info())

print(winners.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          950 non-null    object
 1   firstname   950 non-null    object
 2   surname     921 non-null    object
 3   motivation  950 non-null    object
 4   share       950 non-null    object
 5   year        950 non-null    object
 6   category    950 non-null    object
dtypes: object(7)
memory usage: 52.1+ KB
None
    id   firstname      surname  \
0  976        John   Goodenough   
1  977  M. Stanley  Whittingham   
2  978       Akira      Yoshino   
3  982     Abhijit     Banerjee   
4  983      Esther        Duflo   

                                          motivation share  year   category  
0     "for the development of lithium-ion batteries"     3  2019  chemistry  
1     "for the development of lithium-ion batteries"     3  2019  chemistry  
2     "for the development of lithium-ion bat

終於完成了！嵌套的數據已被正規化並轉換成二維表結構中。

### 將DataFrame導出成JSON檔案文件

現在讓我們嘗試相反的過程：將DataFrame轉換為JSON表示並將其導出成JSON文件檔案。 `to_json`方法從pandas數據結構創建`JSON`字串。它的`orient`參數定義pandas返回數據的格式。下一個範例使用參數`records`返回鍵值對象的JSON陣列。 Pandas將列名稱存儲為字典`鍵`，指向該行的相應值。

In [13]:
print(winners.head(2))

winners_json = winners.head(2).to_json(orient='records')

# 引入 json 模組來解析json字串並打印出來
import json

print(json.dumps(json.loads(winners_json), indent=2))

    id   firstname      surname  \
0  976        John   Goodenough   
1  977  M. Stanley  Whittingham   

                                       motivation share  year   category  
0  "for the development of lithium-ion batteries"     3  2019  chemistry  
1  "for the development of lithium-ion batteries"     3  2019  chemistry  
[
  {
    "id": "976",
    "firstname": "John",
    "surname": "Goodenough",
    "motivation": "\"for the development of lithium-ion batteries\"",
    "share": "3",
    "year": "2019",
    "category": "chemistry"
  },
  {
    "id": "977",
    "firstname": "M. Stanley",
    "surname": "Whittingham",
    "motivation": "\"for the development of lithium-ion batteries\"",
    "share": "3",
    "year": "2019",
    "category": "chemistry"
  }
]


pandas的`to_json()`方法中的參數`orient`包含了幾種Dataframe轉成JSON格式的選項:
* `split` : dict like {‘index’ -> [index], ‘columns’ -> [columns], ‘data’ -> [values]}
* records
* index
* columns
* values
* table


參考: [pandas.DataFrame.to_json](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_json.html)

JSON格式符合你的期望後，將JSON文件名作為第一個參數傳遞給`to_json`方法。Pandas會將JSON字串寫入與Jupyter Notebook相同目錄中的JSON文件中。

In [14]:
winners.to_json("winners.json", orient = "records")

## 讀取和寫入CSV文件

我們的下一個數據集是美國紐約市的嬰兒名字的集合。每筆資料包括姓名`name`，出生年月`birth year`，性別`gender`，種族`ethnicity`，人數`count`和受歡迎程度`popularity rank`。 CSV文件託管在紐約市政府網站上，並位於 https://data.cityofnewyork.us/api/views/25th-nujf/rows.csv。

### 將CSV文件加載到DataFrame中

我們可以在Web瀏覽器中訪問該網站，並將數據集下載到我們的計算機上以進行本地存儲(data/Popular_Baby_Names.csv)。或者，我們也可以將URL作為第一個參數直接傳遞給read_csv函數。Pandas將從網站上獲取數據集。當託管數據頻繁更改時，透過URL來取得更新的數據會很有幫助。它為我們節省了下載數據集的手動工作。

In [15]:
baby_names = pd.read_csv('data/Popular_Baby_Names.csv')

print(baby_names.info())

print(baby_names.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29464 entries, 0 to 29463
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Year of Birth       29464 non-null  int64 
 1   Gender              29464 non-null  object
 2   Ethnicity           29464 non-null  object
 3   Child's First Name  29464 non-null  object
 4   Count               29464 non-null  int64 
 5   Rank                29464 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 1.3+ MB
None
   Year of Birth  Gender Ethnicity Child's First Name  Count  Rank
0           2011  FEMALE  HISPANIC          GERALDINE     13    75
1           2011  FEMALE  HISPANIC                GIA     21    67
2           2011  FEMALE  HISPANIC             GIANNA     49    42
3           2011  FEMALE  HISPANIC            GISELLE     38    51
4           2011  FEMALE  HISPANIC              GRACE     36    53


### 將DataFrame導出成CSV檔案文件

讓我們嘗試使用`to_csv()`方法將baby_names的dataframe物件導出成為CSV文件。若不帶參數的情況下，該方法將直接在Jupyter Notebook中輸出CSV字串。遵循CSV約定，pandas用`逗號`分隔行值，並使用換行符分隔不同的行。提醒一下，`\n`字符表示Python中的換行符。以下是該方法輸出的一個資料預覽。

In [16]:
result_csv = baby_names.head(10).to_csv()

print(result_csv)

,Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank
0,2011,FEMALE,HISPANIC,GERALDINE,13,75
1,2011,FEMALE,HISPANIC,GIA,21,67
2,2011,FEMALE,HISPANIC,GIANNA,49,42
3,2011,FEMALE,HISPANIC,GISELLE,38,51
4,2011,FEMALE,HISPANIC,GRACE,36,53
5,2011,FEMALE,HISPANIC,GUADALUPE,26,62
6,2011,FEMALE,HISPANIC,HAILEY,126,8
7,2011,FEMALE,HISPANIC,HALEY,14,74
8,2011,FEMALE,HISPANIC,HANNAH,17,71
9,2011,FEMALE,HISPANIC,HAYLEE,17,71



注意字符串開頭的逗號和每個`\n`符號後面的數值（0、1、2等）。預設情況下，pandas在CSV字符串中包含DataFrame`索引`列。我們可以通過將`index`參數傳遞為`False`來排除索引列的導出。

In [17]:
result_csv = baby_names.head(10).to_csv(index=False)

print(result_csv)

Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank
2011,FEMALE,HISPANIC,GERALDINE,13,75
2011,FEMALE,HISPANIC,GIA,21,67
2011,FEMALE,HISPANIC,GIANNA,49,42
2011,FEMALE,HISPANIC,GISELLE,38,51
2011,FEMALE,HISPANIC,GRACE,36,53
2011,FEMALE,HISPANIC,GUADALUPE,26,62
2011,FEMALE,HISPANIC,HAILEY,126,8
2011,FEMALE,HISPANIC,HALEY,14,74
2011,FEMALE,HISPANIC,HANNAH,17,71
2011,FEMALE,HISPANIC,HAYLEE,17,71



要將字符串寫入CSV文件，請將其文件名作為第一個參數傳遞給該方法。確保包括.csv擴展名。如果我們不提供特定的路徑，Pandas會將文件寫入Jupyter Notebook所在的目錄。

In [18]:
baby_names.to_csv('nyc_baby_names.csv', index=False)

該方法在Notebook單元下方不會產生任何輸出。但是，如果我們回到Jupyter Notebook導航界面，則可以看到Pandas已經創建了CSV文件檔案了。

預設情況下，pandas將所有DataFrame列寫入CSV文件。我們可以通過將列名稱的子集傳遞給columns參數來限制列的導出。下一個範例生成僅包含 Gender, Child's First Name及Count列的CSV。請注意，執行Jupyter Notebook單元會覆蓋目錄中現有的`nyc_baby_names.csv`文件。

In [19]:
baby_names.to_csv('nyc_baby_names.csv',
                 index=False,
                 columns=["Gender", "Child's First Name", "Count"])

如果文字檔是使用 `\t` (TAB)來做為欄位的分隔字符的資料檔(TSV), 只要`read_csv()`或`write_csv()`指定`sep='\t'`就可以處理。

## 讀取和寫入Excel工作簿

Excel是當今使用最廣泛的電子表格應用程序。通過Pandas，可以輕鬆地讀取和寫入Excel工作簿，甚至特定的工作表。

Pandas需要`xlrd`和`openpyxl`套件才能與Excel交互。這些套件是將Python連接到Excel的粘合劑。

參考: https://pypi.org/project/xlrd/

參考: https://pypi.org/project/openpyxl/

### 將XLSX文件加載到DataFrame中

Pandas的`read_excel()`函數將Excel工作簿導入到DataFrame中。它的第一個參數`io`接受帶有工作簿路徑的字符串。確保在文件名中包含`xlsx`擴展名。預設情況下，panbdas僅導入工作簿中的第一個工作表。

範例資料檔 `Single Worksheet.xlsx` Excel工作簿是一個不錯的起點。它僅包含一個`Data`工作表。

![](images/12_06.png)

In [22]:
df_xlsx = pd.read_excel('data/Single Worksheet.xlsx')

print(df_xlsx.info())

print(df_xlsx.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   First Name  5 non-null      object
 1   Last Name   5 non-null      object
 2   City        5 non-null      object
 3   Gender      5 non-null      object
dtypes: object(4)
memory usage: 288.0+ bytes
None
  First Name Last Name           City Gender
0    Brandon     James          Miami      M
1       Sean   Hawkins         Denver      M
2       Judy       Day    Los Angeles      F
3     Ashley      Ruiz  San Francisco      F
4  Stephanie     Gomez       Portland      F


`read_excel()`函數支持許多與`read_csv()`相同的參數，包括用於設置索引列的`index_col`，用於選擇列的`usecols`以及用於將單列工作表強制轉換為`Series`對象的`usecols`。在下一個範例中，我們將**City**列設置為索引，並僅保留數據集的4列中的3列。注意，如果我們為index_col參數提供一列，則還必須將該列包括在usecols列表中。

In [24]:
df_xlsx = pd.read_excel(io='data/Single Worksheet.xlsx',
                       usecols=["City", "First Name", "Last Name"],
                       index_col="City")

print(df_xlsx.info())

print(df_xlsx.head())

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, Miami to Portland
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   First Name  5 non-null      object
 1   Last Name   5 non-null      object
dtypes: object(2)
memory usage: 120.0+ bytes
None
              First Name Last Name
City                              
Miami            Brandon     James
Denver              Sean   Hawkins
Los Angeles         Judy       Day
San Francisco     Ashley      Ruiz
Portland       Stephanie     Gomez


當一個工作簿包含多個工作表時，複雜度會稍微增加。 `Multiple Worksheets.xlsx`文件包含三個工作表：Data1，Data2和Data3。預設情況下，pandas僅導入工作簿中的第一個工作表。

![](images/12_07.png)

![](images/12_08.png)

![](images/12_09.png)

In [26]:
df_xlsx = pd.read_excel("data/Multiple Worksheets.xlsx")

print(df_xlsx.info())

print(df_xlsx.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   First Name  5 non-null      object
 1   Last Name   5 non-null      object
 2   City        5 non-null      object
 3   Gender      5 non-null      object
dtypes: object(4)
memory usage: 288.0+ bytes
None
  First Name Last Name           City Gender
0    Brandon     James          Miami      M
1       Sean   Hawkins         Denver      M
2       Judy       Day    Los Angeles      F
3     Ashley      Ruiz  San Francisco      F
4  Stephanie     Gomez       Portland      F


Pandas為每個工作表分配一個從0開始的索引。要導入特定的工作表，我們可以將`sheet_name`參數傳遞給工作表的索引位置或其名稱。預設參數為`0`（第一個工作表）。

In [28]:
df_xlsx = pd.read_excel("data/Multiple Worksheets.xlsx", sheet_name=0)

df_xlsx = pd.read_excel("data/Multiple Worksheets.xlsx", sheet_name='Data 1')

print(df_xlsx.info())

print(df_xlsx.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   First Name  5 non-null      object
 1   Last Name   5 non-null      object
 2   City        5 non-null      object
 3   Gender      5 non-null      object
dtypes: object(4)
memory usage: 288.0+ bytes
None
  First Name Last Name           City Gender
0    Brandon     James          Miami      M
1       Sean   Hawkins         Denver      M
2       Judy       Day    Los Angeles      F
3     Ashley      Ruiz  San Francisco      F
4  Stephanie     Gomez       Portland      F


要導入所有工作表，請將參數`None`傳遞給`sheet_name`參數。 `read_excel()`函數返回一個字典dict物件，其中工作表的名稱為`鍵`，而相應的DataFrames為`值`。

In [29]:
dict_xlsx = pd.read_excel('data/Multiple Worksheets.xlsx', sheet_name=None)

print(type(dict_xlsx))

print(dict_xlsx)

<class 'dict'>
{'Data 1':   First Name Last Name           City Gender
0    Brandon     James          Miami      M
1       Sean   Hawkins         Denver      M
2       Judy       Day    Los Angeles      F
3     Ashley      Ruiz  San Francisco      F
4  Stephanie     Gomez       Portland      F, 'Data 2':   First Name Last Name           City Gender
0     Parker     Power        Raleigh      F
1    Preston  Prescott   Philadelphia      F
2    Ronaldo   Donaldo         Bangor      M
3      Megan   Stiller  San Francisco      M
4     Bustin    Jieber         Austin      F, 'Data 3':   First Name  Last Name     City Gender
0     Robert     Miller  Seattle      M
1       Tara     Garcia  Phoenix      F
2    Raphael  Rodriguez  Orlando      M}


要訪問一個DataFrame/工作表，我們訪問字典中的一個`鍵`。

In [31]:
df_xlsx = dict_xlsx["Data 2"]

print(df_xlsx.info())

print(df_xlsx.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   First Name  5 non-null      object
 1   Last Name   5 non-null      object
 2   City        5 non-null      object
 3   Gender      5 non-null      object
dtypes: object(4)
memory usage: 288.0+ bytes
None
  First Name Last Name           City Gender
0     Parker     Power        Raleigh      F
1    Preston  Prescott   Philadelphia      F
2    Ronaldo   Donaldo         Bangor      M
3      Megan   Stiller  San Francisco      M
4     Bustin    Jieber         Austin      F


要限制pandas導入的工作表範圍，請將`sheet_name`參數傳遞給索引位置或工作表名稱的列表。Pandas將返回一個字典，該字典的鍵與sheet_name列表中的字符串匹配。下一個範例僅導入Data 1和Data 3工作表。

In [35]:
dict_xlsx = pd.read_excel('data/Multiple Worksheets.xlsx',
                         sheet_name=["Data 1", "Data 3"])

print(type(dict_xlsx))

print(dict_xlsx)

<class 'dict'>
{'Data 1':   First Name Last Name           City Gender
0    Brandon     James          Miami      M
1       Sean   Hawkins         Denver      M
2       Judy       Day    Los Angeles      F
3     Ashley      Ruiz  San Francisco      F
4  Stephanie     Gomez       Portland      F, 'Data 3':   First Name  Last Name     City Gender
0     Robert     Miller  Seattle      M
1       Tara     Garcia  Phoenix      F
2    Raphael  Rodriguez  Orlando      M}


### 將DataFrame導出成XSLX檔案文件

讓我們使用`baby_names`的DataFrame物件來作為Excel導出的資料集。

In [36]:
print(baby_names.info())

print(baby_names.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29464 entries, 0 to 29463
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Year of Birth       29464 non-null  int64 
 1   Gender              29464 non-null  object
 2   Ethnicity           29464 non-null  object
 3   Child's First Name  29464 non-null  object
 4   Count               29464 non-null  int64 
 5   Rank                29464 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 1.3+ MB
None
   Year of Birth  Gender Ethnicity Child's First Name  Count  Rank
0           2011  FEMALE  HISPANIC          GERALDINE     13    75
1           2011  FEMALE  HISPANIC                GIA     21    67
2           2011  FEMALE  HISPANIC             GIANNA     49    42
3           2011  FEMALE  HISPANIC            GISELLE     38    51
4           2011  FEMALE  HISPANIC              GRACE     36    53


假設我們想將數據集分為2個DataFrame，每種性別單獨一個。然後，我們想將每個DataFrame寫入到一個新的Excel工作簿中的成為單獨的工作表。

我們可以從過濾baby_names的DataFrame開始。

In [37]:
girls = baby_names[baby_names["Gender"] == "FEMALE"]
boys = baby_names[baby_names["Gender"] == "MALE"]

與寫入CSV相比，寫入Excel工作簿需要更多的步驟。首先，我們需要創建一個`ExcelWriter`物件。該物件是工作簿的基礎。我們將各個`工作表`附加到該物件上。

`ExcelWriter`構造函數的第一個參數`path`接受帶有新工作簿文件名的字符串。如果我們不提供路徑，pandas將在與Jupyter Notebook相同的目錄中創建Excel文件。

In [38]:
excel_file = pd.ExcelWriter("baby_names.xlsx")

接下來，我們需要將我們的DataFrame物件連接到工作簿中的各個工作表。 DataFrame的`to_excel()`方法的第一個參數`excel_writer`接受`ExcelWriter`物件。 `sheet_name`參數接受具有所需工作表名稱的字符串。

最後，我們可以將`index`參數傳遞False值來排除DataFrame索引。

In [39]:
girls.to_excel(excel_writer=excel_file,
              sheet_name="Girls",
              index=False)

請注意，此時pandas尚未創建Excel工作簿。

讓我們將第二個DataFrame連接到Excel工作簿。我們在boys的DataFrame上調`to_excel()`方法，並將相同的ExcelWriter對像傳遞給excel_writer參數。現在，pandas知道應該將兩個數據集寫入同一工作簿。
    
我們還要將`sheet_name`參數的字符串參數更改為其他名稱。要過濾大熊貓將包括的列，請將其名稱列表傳遞給columns參數。下一個範例告訴pandas只包括Child's First Name, Count 和 Rank列。

In [40]:
boys.to_excel(
             excel_writer=excel_file,
             sheet_name = "Boys",
             index = False,
             columns = ["Child's First Name", "Count", "Rank"]
         )

現在，我們已經配置了好創建Excel工作簿的相關資料，現在可以將其寫入檔案系統了。在`excel_file`的ExcelWriter物件上調用`save()`方法來完成最後的步驟。

In [41]:
excel_file.save()

查看Jupyter Notebook界面；pandas在同一文件夾中創建了`baby_names.xlsx`文件。

## 讀取和寫入Parquet文件


![](images/parquet_01_row_column.png)

![](images/parquet_02_row_column.png)

**範例數據**: `nyc_taxi_dataset.zip`檔案是`紐約市計程車和禮車協會`所發佈的一個公開資料集。

資料筆數: 729,322 （73萬筆）
資料格式: CSV
資料大小(未壓縮): 94 MB
資料大小(己壓縮): 32 MB

資料欄位:
* id - 車程編號
* vendor_id - 計程車行編號
* pickup_datetime - 上車日期時間
* dropoff_datetime - 下車日期時間
* passenger_count - 乘客人數
* pickup_longitude - 上車經度
* pickup_latitude - 上車緯度
* dropoff_longitude - 下車經度
* dropoff_latitude - 下車緯度
* store_and_fwd_flag - 是否直接連線上傳b Y/N
* trip_duration - 車程持續時間(以秒為單位)


### 將CSV文件加載到DataFrame中並導出成Parquet文件

In [1]:
import pandas as pd

nyc_tax = pd.read_csv('data/nyc_taxi_dataset.zip')

print(nyc_tax.info())
print(nyc_tax.head())
print(nyc_tax.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  729322 non-null  object 
 1   vendor_id           729322 non-null  int64  
 2   pickup_datetime     729322 non-null  object 
 3   dropoff_datetime    729322 non-null  object 
 4   passenger_count     729322 non-null  int64  
 5   pickup_longitude    729322 non-null  float64
 6   pickup_latitude     729322 non-null  float64
 7   dropoff_longitude   729322 non-null  float64
 8   dropoff_latitude    729322 non-null  float64
 9   store_and_fwd_flag  729322 non-null  object 
 10  trip_duration       729322 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 61.2+ MB
None
          id  vendor_id      pickup_datetime     dropoff_datetime  \
0  id1080784          2  2016-02-29 16:40:21  2016-02-29 16:47:01   
1  id0889885          1  2016-03-

使用Pandas來轉換Dataframe成為`parquet`格式並且進行壓縮。

資料筆數: 729,322 （73萬筆）
資料格式: parquet
資料大小(未壓縮): 26 MB
資料大小(己壓縮): 18 MB    

In [2]:
nyc_tax.to_parquet('data/nyc_taxi_trip_duration.parquet')

nyc_tax.to_parquet('data/nyc_taxi_trip_duration.parquet.gz', compression='gzip')

In [8]:
nyc_tax['pickup_datetime']

0         2016-02-29 16:40:21
1         2016-03-11 23:35:37
2         2016-02-21 17:59:33
3         2016-01-05 09:44:31
4         2016-02-17 06:42:23
                 ...         
729317    2016-05-21 13:29:38
729318    2016-02-22 00:43:11
729319    2016-04-15 18:56:48
729320    2016-06-19 09:50:47
729321    2016-01-01 17:24:16
Name: pickup_datetime, Length: 729322, dtype: object

### 將Parquet文件加載到DataFrame中

In [4]:
nyc_tax = pd.read_parquet('data/nyc_taxi_trip_duration.parquet')

print(nyc_tax.info())
print(nyc_tax.head())
print(nyc_tax.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  729322 non-null  object 
 1   vendor_id           729322 non-null  int64  
 2   pickup_datetime     729322 non-null  object 
 3   dropoff_datetime    729322 non-null  object 
 4   passenger_count     729322 non-null  int64  
 5   pickup_longitude    729322 non-null  float64
 6   pickup_latitude     729322 non-null  float64
 7   dropoff_longitude   729322 non-null  float64
 8   dropoff_latitude    729322 non-null  float64
 9   store_and_fwd_flag  729322 non-null  object 
 10  trip_duration       729322 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 61.2+ MB
None
          id  vendor_id      pickup_datetime     dropoff_datetime  \
0  id1080784          2  2016-02-29 16:40:21  2016-02-29 16:47:01   
1  id0889885          1  2016-03-